# En este Notebook se encuentran los pasos necesarios para crear los diccionarios y fastas de los transcritos del pipeline de Bell, para A. thaliana.

Solo es necesario tener los siguientes archivos:  
Araport11_GTF_genes_transposons.current.gtf **(ARAPORT11)**  
Araport11_upstream_3000_20220914.gz **(ARAPORT11)**  
Araport11_downstream_3000_20220914.gz **(ARAPORT11)**  
Araport11_intergenic_20220914.gz **(ARAPORT11)**  
Araport11_intron_20220914.gz **(ARAPORT11)**  
LncRNA_Anotacion_Lab.tsv **(FIGSHARE, lab)**

**TAIR10.1**  
wget --no-check-certificate "https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/001/735/GCF_000001735.4_TAIR10.1/GCF_000001735.4_TAIR10.1_genomic.fna.gz"   
**ARAPORT11 GTF**    
wget --no-check-certificate "https://www.arabidopsis.org/download_files/Genes/Araport11_genome_release/Araport11_GFF3_genes_transposons.201606.gff.gz"  
**downstream_3000**  
wget --no-check-certificate "https://www.arabidopsis.org/download_files/Sequences/Araport11_blastsets/Araport11_downstream_3000_20220914.gz"  
**upstream_3000**  
wget --no-check-certificate "https://www.arabidopsis.org/download_files/Sequences/Araport11_blastsets/Araport11_upstream_3000_20220914.gz"  
**intergenic**  
wget --no-check-certificate "https://www.arabidopsis.org/download_files/Sequences/Araport11_blastsets/Araport11_intergenic_20220914.gz"  
**intron**  
wget --no-check-certificate "https://www.arabidopsis.org/download_files/Sequences/Araport11_blastsets/Araport11_intron_20220914.gz"  
**LncRNA**  
wget --no-check-certificate "https://figshare.com/ndownloader/files/36181233?private_link=5d19c00590cbd7fe73a1"


Si se tiene alguna duda se puede consultar el notebook "Archivos_dic.ipynb".

Del GTF se van a generar los diccionarios de los siguientes transcritos:  
**three_prime_UTR**  
**five_prime_UTR**  
**exon**    
**miRNA**  
**mRNA**  
**ncRNA**  
**rRNA**  
**snoRNA**  
**snRNA**  
**tRNA**  
**gene**

loop para crear todos los diccionarios del GTF.

In [1]:
%%bash
cd ../Dict
#for PRO in three_prime_UTR five_prime_UTR exon lnc_RNA miRNA mRNA ncRNA rRNA snoRNA snRNA tRNA; do awk -F'\t' -v PRO="$PRO" 'BEGIN{OFS="\t"} $3==PRO {gsub(/"/, "", $9);split($9, arr, "; "); col1=arr[1]; col2=$3; col3=$1":"$4".."$5; col4=$1; col5=$4; col6=$5; col7=$7; col8="."; col9=arr[2]; col10=arr[2]; col11=arr[2]; col12=($4>$5)?($4-$5):($5-$4); if (col7 == "+") { print col1, col2, col3, col4, col5, col6, col7, col8, col9, col10, col11, col12 } else { print col1, col2, col3, col4, col6, col5, col7, col8, col9, col10, col11, col12 }}' Araport11_GTF_genes_transposons.current.gtf | sed -e 's/;//g' -e 's/transcript_id//g' -e 's/gene_id//g' | awk '{$1=$1};1' OFS='\t'| head -n 5; done
### Ver si los print son correctos
#for PRO in three_prime_UTR five_prime_UTR exon miRNA mRNA ncRNA rRNA snoRNA snRNA tRNA; do awk -F'\t' -v PRO="$PRO" 'BEGIN{OFS="\t"} $3==PRO {gsub(/"/, "", $9);split($9, arr, "; "); col1=arr[1]; col2=$3; col3=$1":"$4".."$5; col4=$1; col5=$4; col6=$5; col7=$7; col8="."; col9=arr[2]; col10=arr[2]; col11=arr[2]; col12=($4>$5)?($4-$5):($5-$4); if (col7 == "+") { print col1, col2, col3, col4, col5, col6, col7, col8, col9, col10, col11, col12 } else { print col1, col2, col3, col4, col6, col5, col7, col8, col9, col10, col11, col12 }}' Araport11_GTF_genes_transposons.current.gtf | sed -e 's/;//g' -e 's/transcript_id//g' -e 's/gene_id//g' | awk '{$1=$1};1' OFS='\t' > ../DMEL_COORDS/"$PRO.coords.dict"; done
### El comando superior le faltaba ver si las anotaciones tenian un tamaño mayor a 0.
 for PRO in three_prime_UTR five_prime_UTR exon gene miRNA mRNA ncRNA rRNA snoRNA snRNA tRNA; do awk -F'\t' -v PRO="$PRO" 'BEGIN{OFS="\t"} $3==PRO && $4!=$5 { gsub(/"/, "", $9); split($9, arr, "; "); col1=arr[1]; col2=$3; col3=$1":"$4".."$5; col4=$1; col5=$4; col6=$5; col7=$7; col8="."; col9=arr[2]; col10=arr[2]; col11=arr[2]; col12=($4>$5)?($4-$5):($5-$4); if (col7 == "+") { print col1, col2, col3, col4, col5, col6, col7, col8, col9, col10, col11, col12 } else { print col1, col2, col3, col4, col6, col5, col7, col8, col9, col10, col11, col12 }}' Araport11_GTF_genes_transposons.current.gtf | sed -e 's/;//g' -e 's/transcript_id//g' -e 's/gene_id//g' | awk '{$1=$1};1' OFS='\t' > ../DMEL_COORDS/"$PRO.coords.dict"; done

De los archivos descargados de ARAPORT11 se van a generar los siguientes transcritos:  
**upstream_3000**  
**downstream_3000**  
**intergenic**  
**intron**

**upstream_3000**

In [2]:
%%bash
cd ../Dict
grep '>' Araport11_upstream_3000_2022091.fasta | sed -e 's/|//g' -e 's/>//g' -e 's/LENGTH=//g' | awk '{ gsub("FORWARD", "+"); gsub("REVERSE", "-"); gsub("-", "..", $2); col3 = substr($2, index($2,":")+1, index($2,"..")-index($2,":")-1); col4 = substr($2, index($2,"..")+2); print $1, substr($2, 1, index($2, ":")-1), col3, col4, $2, $3, $4 }' OFS='\t' | awk -v OFS='\t' '{ if ($6 == "-") { temp = $5; $5 = $6; $6 = temp; print $1, "upstream_3000", $6, $2, $3, $4, $5, ".", $1, $1, $1, $7 } else { print $1, "upstream_3000", $5, $2, $3, $4, $6, ".", $1, $1, $1, $7 } }' | awk -F'\t' 'BEGIN{OFS="\t"} { if ($7 == "-") { temp = $5; $5 = $6; $6 = temp } print $1, $2, $3, $4, $5, $6, $7, $8, $9, $10, $11, $12 }' > ../DMEL_COORDS/upstream_3000.coords.dict 


**downstream_3000**

In [3]:
%%bash
cd ../Dict
grep '>' Araport11_downstream_3000_2022091.fasta | sed -e 's/|//g' -e 's/>//g' -e 's/LENGTH=//g' | awk '{ gsub("FORWARD", "+"); gsub("REVERSE", "-"); gsub("-", "..", $2); col3 = substr($2, index($2,":")+1, index($2,"..")-index($2,":")-1); col4 = substr($2, index($2,"..")+2); print $1, substr($2, 1, index($2, ":")-1), col3, col4, $2, $3, $4 }' OFS='\t' | awk -v OFS='\t' '{ if ($6 == "-") { temp = $5; $5 = $6; $6 = temp; print $1, "downstream_3000", $6, $2, $3, $4, $5, ".", $1, $1, $1, $7 } else { print $1, "downstream_3000", $5, $2, $3, $4, $6, ".", $1, $1, $1, $7 } }' | awk -F'\t' 'BEGIN{OFS="\t"} { if ($7 == "-") { temp = $5; $5 = $6; $6 = temp } print $1, $2, $3, $4, $5, $6, $7, $8, $9, $10, $11, $12 }' > ../DMEL_COORDS/downstream_3000.coords.dict

**intron**

In [4]:
%%bash
cd ../Dict
grep '>' Araport11_intron_2022091.fasta | sed -e 's/|//g' -e 's/>//g' -e 's/LENGTH=//g' | awk '{$1=$1};1' OFS='\t'|cut -f1,3,4,5| awk '{ gsub("FORWARD", "+"); gsub("REVERSE", "-"); gsub("-", "..", $2); col3 = substr($2, index($2,":")+1, index($2,"..")-index($2,":")-1); col4 = substr($2, index($2,"..")+2); print $1, substr($2, 1, index($2, ":")-1), col3, col4, $2, $3, $4 }' OFS='\t' | awk -v OFS='\t' '{ if ($6 == "-") { temp = $5; $5 = $6; $6 = temp; print $1, "intron", $6, $2, $3, $4, $5, ".", $1, $1, $1, $7 } else { print $1, "intron", $5, $2, $3, $4, $6, ".", $1, $1, $1, $7 } }' | awk -F'\t' 'BEGIN{OFS="\t"} { if ($7 == "-") { temp = $5; $5 = $6; $6 = temp } print $1, $2, $3, $4, $5, $6, $7, $8, $9, $10, $11, $12 }' > ../DMEL_COORDS/intron.coords.dict

**intergenic**

In [5]:
%%bash
cd ../Dict
grep '>' Araport11_intergenic_2022091.fasta | sed -e 's/|//g' -e 's/>//g' -e 's/LENGTH=//g' | awk '{ gsub("FORWARD", "+"); gsub("REVERSE", "-"); gsub("-", "..", $2); col3 = substr($2, index($2,":")+1, index($2,"..")-index($2,":")-1); col4 = substr($2, index($2,"..")+2); print $1, substr($2, 1, index($2, ":")-1), col3, col4, $2, $3, $4 }' OFS='\t' | awk -v OFS='\t' '{ if ($6 == "-") { temp = $5; $5 = $6; $6 = temp; print $1, "intergenic", $6, $2, $3, $4, $5, ".", $1, $1, $1, $7 } else { print $1, "intergenic", $5, $2, $3, $4, $6, ".", $1, $1, $1, $7 } }' | awk -F'\t' 'BEGIN{OFS="\t"} { if ($7 == "-") { temp = $5; $5 = $6; $6 = temp } print $1, $2, $3, $4, $5, $6, $7, $8, $9, $10, $11, $12 }' > ../DMEL_COORDS/intergenic.coords.dict


Del archivo descargado de figshare, se generara el diccionario para los lncRNA

**LncRNA**

In [6]:
%%bash
cd ../Dict
cut -f1,2,3,4,5,7,17 LncRNA_Anotacion_Lab.tsv|awk 'NR > 1'|awk -v OFS='\t' '{print $4, $7, $1":"$2".."$3, $1, $2, $3, $6, ".", $5, $5, $5, ($2>$3)?($2-$3):($3-$2)}'|awk -F'\t' 'BEGIN{OFS="\t"} { if ($7 == "-") { temp = $5; $5 = $6; $6 = temp } print $1, $2, $3, $4, $5, $6, $7, $8, $9, $10, $11, $12 }' > ../DMEL_COORDS/lncRNA.coords.dict 


## FASTA

Generar los archivos .bed del GTF

In [2]:
%%bash
cd ../Dict
for PRO in three_prime_UTR five_prime_UTR exon gene miRNA mRNA ncRNA rRNA snoRNA snRNA tRNA; do awk -F'\t' -v PRO="$PRO" 'BEGIN{OFS="\t"} $3==PRO' Araport11_GTF_genes_transposons.current.gtf | cut -f1,4,5,7,8,9 | awk -F'\t' -v OFS='\t' '{print $1, $2, $3, $6, $5, $4}' > "BED_FILES/${PRO}_Araport11.bed"; done

Generar los .fasta del GTF

In [3]:
%%capture
%%bash
cd ../Dict/BED_FILES #(añadir -s BLAT EN UCSC, NO TOMAR EN CUENTA LOS LNC DE ARAPORT) PRIORIDAD VER LA MUESTRA CON EL PIPE DE BELL....
for PRO in *.bed; do bedtools getfasta -fi ../TAIR10_chr_all.fas -bed "$PRO" -s -name -fo "${PRO%.bed}.fasta"; done

In [4]:
%%capture
%%bash
cd ../Dict/BED_FILES
mv *.fasta ../../FASTA_TRA

In [12]:
%%bash
cd ../Dict
cp  Araport11_upstream_3000_2022091.fasta Araport11_intergenic_2022091.fasta Araport11_intron_2022091.fasta Araport11_downstream_3000_2022091.fasta ../FASTA_TRA

Ya se tienen los fasta de Araport11, por lo que solo falta generar el tsv de los lncRNA.

Bed del LncRNA.

In [13]:
%%bash
cd ../Dict
cut -f1,2,3,4,5,7 LncRNA_Anotacion_Lab.tsv|awk 'NR > 1'|awk -v OFS='\t' '{print $1, $2, $3, "Transcript_ID; "$4";" "Gene_ID; "$5, ".", $6}'> BED_FILES/LncRNA_Anotacion_Lab.bed

Fasta del LncRNA.

In [14]:
%%capture
%%bash
cd ../Dict/BED_FILES
bedtools getfasta -fi ../TAIR10_chr_all.fas -bed LncRNA_Anotacion_Lab.bed -s -name -fo LncRNA_Anotacion_Lab.fasta

In [15]:
%%capture
%%bash
cd ../Dict/BED_FILES
mv LncRNA_Anotacion_Lab.fasta ../../FASTA_TRA